In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # Берём должников у которых нужно посмотреть

err_not_final = 'в последнем скаченном АСИОИП файле нет "итога"'
dict_log_pas = {"kpi":{'log':'901021350973_210540032049', 'pas':'Kk12345%'}} # kpi

loading_path_all = r"C:\Users\User\Desktop\py scripts\АИСОИП отмены kpi"
loading_path_state_load = r"C:\Users\User\Desktop\py scripts\АИСОИП статусы kpi"



from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


import sqlite3
from pathlib import Path
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from tqdm import tqdm
import time
import pandas as pd
from bs4 import BeautifulSoup
import os
import pickle
import math
from selenium.webdriver.support import expected_conditions as EC

from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import date
from datetime import datetime, timedelta
import numpy as np

import sys
import traceback
from selenium.webdriver.support.wait import WebDriverWait
from dateutil.relativedelta import relativedelta

import os
import shutil

from collections import defaultdict
import PyPDF2
from pathlib import Path
import pandas as pd
import re





path_check_iin = Path(loading_path_all).joinpath('Проверенные ранее.xlsx')
load_path = Path(loading_path_state_load)
file_date = {}

for f in load_path.iterdir():
    
    if f.is_dir():
        p_name = f.name

        p_name_s = p_name.split(' ')
        p_time = p_name_s[1]

        dt = datetime.strptime(p_time, "%Y-%m-%d___%H-%M")
        file_date[f] = dt

        
new_folder = max(file_date, key=file_date.get)


# In[2]:


itog_file = ''
for f in new_folder.iterdir(): 
    if 'google_table' in f.name.lower() and '$' not in f.name.lower():
        itog_file = f
    
        
if itog_file == '':
    print(err_not_final)


# In[3]:


df_main_ = pd.read_excel(itog_file, dtype='str')[['Номер_строки_google_t', '№', 'Номер исполнительного производства', 'ИИН/БИН должника', 'Статус исполнительного производства', 'Судебный исполнитель']]

# In[4]:


df_main_ = df_main_[df_main_['Статус исполнительного производства'] == 'Окончено']

df_main_start_len = len(df_main_)
# # Убираю ранее проверенные ИИН

# In[5]:


df_old_ = pd.read_excel(path_check_iin, dtype='str')
df_old_['ИИН'] = df_old_['ИИН'].apply(lambda x: (12 - len(str(x)))*'0'+ str(x) )
df_main_['ИИН/БИН должника'] = df_main_['ИИН/БИН должника'].apply(lambda x: (12 - len(str(x)))*'0'+ str(x))

df_old_['ИИН_and_id_Испол'] = df_old_['ИИН'] + '_' + df_old_['Номер исполнительного производства']
df_main_['ИИН_and_id_Испол'] = df_main_['ИИН/БИН должника'] + '_' + df_main_['Номер исполнительного производства']


df_main_ = df_main_[~df_main_['ИИН_and_id_Испол'].isin(df_old_['ИИН_and_id_Испол'])].copy()



# # Создаю папку для скачивания

# In[6]:


error_list = []
result_list = []


now = datetime.now()
dt_string = now.strftime("%Y-%m-%d___%H-%M")


loading_path = f'{loading_path_all}/new_load'
loading_path_docs = f'{loading_path}/docs'
loading_path_db = f'{loading_path}/Отмены_.db'

if not os.path.exists(loading_path):
    os.mkdir(loading_path)
    os.mkdir(loading_path_docs)
    
conn_local_db = sqlite3.connect(loading_path_db)

    
try:
    conn_local_db = sqlite3.connect(loading_path_db)
    query = ''' select * from cancels_ '''
    df_res_old = pd.read_sql_query(query, conn_local_db)
    
    # df_res_old = df_res_old.loc[df_res_old['найдено строк'] != 'error']   #Ошибка не оставляю.
    df_res_old['ИИН'] = df_res_old['ИИН'].apply(lambda x: (12-len(str(x)))*'0'+str(x))
except:
    df_res_old = pd.DataFrame(columns=['ИИН', 'Номер исполнительного производства'])


    
## Убираю старые значения значения.
if df_res_old.size > 0:
    df_res_old_copy = df_res_old.copy()
    df_res_old_copy['bool'] = 1
    df_main__copy = df_main_.merge(df_res_old_copy[['Номер исполнительного производства', 'ИИН', 'bool']],
                                  right_on=['Номер исполнительного производства', 'ИИН'], 
                                  left_on=['Номер исполнительного производства', 'ИИН/БИН должника'], 
                                  how='left')
    
    df_main_ = df_main__copy[df_main__copy['bool'].isna()]
    df_res_old_copy = None
    df_main_.drop(['bool', 'ИИН'], axis=1, inplace=True)
    
#### 
#### TODO


# ## Импорт


def table_exists(conn, table_name):
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
    result = cursor.fetchone()
    cursor.close()
    return result is not None

def save_data(result_list_batch,  conn, new_table, table_name = 'cancels_'):
#     print('Созранены данные при')
    col = ['Имя портфеля', 'index', 'eid', 'ЧСИ', 'ИИН',
           'Номер исполнительного производства', 'найдено строк', 
           'найдено дело', 'Кол-во файлов'] # ['ИИН', 'Номер исполнительного производства']
    df_res_new = pd.DataFrame(result_list_batch, columns=col)
    
    if new_table and not table_exists(conn=conn, table_name=table_name):
        df_res_new.to_sql(table_name, conn, if_exists='replace', index=False)
    else:
        df_res_new.to_sql(table_name, conn, if_exists='append', index=False)
    
    result_list_batch = []
    

def to_site(loading_path_all):

    # Вхожу на сайт
    url = 'https://aisoip.adilet.gov.kz/cabinet/exec-productions'

    # Добавляю папку сохранения 
    #https://stackoverflow.com/questions/34515328/how-to-set-default-download-directory-in-selenium-chrome-capabilities
    #https://stackoverflow.com/questions/18026391/selenium-webdriver-in-python-files-download-directory-change-in-chrome-prefere
    
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    prefs = {"profile.default_content_settings.popups": 0,
             "download.default_directory": (loading_path_all + '/').replace('/', '\\'), # IMPORTANT - ENDING SLASH V IMPORTANT
             "directory_upgrade": True,
             "profile.managed_default_content_settings.images": 2}
    options.add_experimental_option("prefs", prefs)
    
    
    # Подключение к сайту
    driver = webdriver.Chrome(options=options)
    # service = Service('C:/Users/User/chromedriver/chromedriver.exe')
    #driver = webdriver.Chrome(service=service, options=options)
    
    driver.implicitly_wait(10)
    driver.get(url)

    time.sleep(2)
    
    return driver
    
    
def write_login(driver, name_login):
    '''Ввожу логин пароль'''
    login_global   = dict_log_pas[name_login]['log']
    pasword_global = dict_log_pas[name_login]['pas']    
    

    login  = login_global
    pasword = pasword_global

    class_field = 'field'
    field = driver.find_elements(By.CLASS_NAME, class_field)[0]
    field = field.find_element(By.CSS_SELECTOR, 'input')
    field.send_keys(login)

    time.sleep(1)
    field = driver.find_elements(By.CLASS_NAME, class_field)[1]
    field = field.find_element(By.CSS_SELECTOR, 'input')
    field.send_keys(pasword)


    class_field = 'ui primary button fluid'.replace(' ', '.')
    field = driver.find_elements(By.CLASS_NAME, class_field)[0].click()

    time.sleep(1)
    
    
def to_targer_sheet_1(driver):
    # Перехожу на нужную вкладку
    # исполнительные производства
    class_field1 = 'cabinet-header'
    class_field2 = 'navigation-item'
    field = driver.find_element(By.CLASS_NAME, class_field1)
    field = field.find_elements(By.CLASS_NAME, class_field2)[1]
    field.click()
    time.sleep(2)
    
def to_targer_sheet_2(driver):
    # Перехожу на нужную вкладку

    class_field1 = 'v-slide-group__content v-tabs-bar__content'.replace(' ', '.')
    class_field2 = 'div'
    field = driver.find_element(By.CLASS_NAME, class_field1)
    field = field.find_elements(By.CSS_SELECTOR, class_field2)[3]
    field.click()
    time.sleep(1)
    
def iin_insert(driver, iin_iter):
    # Вставляю ИИН
    field = driver.find_element(By.CLASS_NAME, 'row mt-8 mt-md-12'.replace(' ', '.'))
    field = field.find_elements(By.CLASS_NAME, 'v-input__slot'.replace(' ', '.'))[0]
    field = field.find_element(By.CSS_SELECTOR, 'input'.replace(' ', '.'))
    field.send_keys(iin_iter)
    time.sleep(0.1)
    
    
    
def buttom_find(driver):
    butom_start_find = '.mb-2 mr-2 mr-md-5 v-btn v-btn--is-elevated v-btn--has-bg theme--light v-size--default primary'.replace(' ', '.')
    field = driver.find_element(By.CSS_SELECTOR, butom_start_find).click()
    time.sleep(3)
    
def count_data_line(driver):
    # Сохраняю Количество данных "из" 
    class_find = 'v-data-footer__pagination'.replace(' ', '.')
    count_line = driver.find_element(By.CLASS_NAME, class_find).text
    time.sleep(0.5)
    return count_line
    
def find_ispol_on_f258(driver, f258_iter):
    '''Найти подходящий испол. И нажимаем на него.'''
    time.sleep(2)
    
    class_find = 'exec-productions-item mb-2 px-3 px-md-5 py-2 py-md-3 v-card v-sheet theme--light'.replace(' ', '.') #Класс самого объекта строки
    obgect_html = driver.find_elements(By.CLASS_NAME, class_find)
    find_f258_iter = 0
    for row in obgect_html:
        class_find = 'col-sm-6 col-lg-3 col-12'.replace(' ', '.') #Класс одной колонки
        field = row.find_elements(By.CLASS_NAME, class_find)[1] # Выбираю вторую колонку/ячейку
        field = field.find_element(By.CLASS_NAME, 'value')
        text_num_ispol = field.text

        if f258_iter in text_num_ispol:
            field.find_element(By.CSS_SELECTOR, "a[style='text-decoration: underline;']").click()
            time.sleep(1)
            find_f258_iter = 1
            break
    return find_f258_iter
        
def find_taken_measures(driver, time_=1):
    time.sleep(time_)
    class_find = 'v-data-table__wrapper'.replace(' ', '.')  # Принятые меры таблица
    field = driver.find_elements(By.CLASS_NAME, class_find)[-1]
    return field.text

def close_detail_window(driver):
    time.sleep(0.5)
    class_find = '.d-print-none.v-btn.v-btn--absolute.v-btn--icon.v-btn--right.v-btn--round.v-btn--top.theme--light.v-size--small'.replace(' ', '.') #Класс самого объекта строки
    field = driver.find_element(By.CSS_SELECTOR, class_find)
    field.click()

def delite_filter(driver):
    time.sleep(0.3)
    
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.v-btn v-btn--outlined theme--light v-size--default'.replace(' ', '.')))
    )

    field.click()
    
def set_100_row(driver):
    # Расширить выдачу до 100
    time.sleep(0.5) 
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.v-input.v-input--hide-details.v-input--is-label-active.v-input--is-dirty.theme--light.v-text-field.v-text-field--is-booted.v-select .v-icon.notranslate.material-icons.theme--light'))
    )

    field.click()
    
    
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.v-list.v-select-list.v-sheet.theme--light.theme--light>:nth-child(4)'))
    )

    field.click()
    flag_100 = False
    
def download_docs(driver, loading_path_docs, eid_iter):
    '''Скачиваю файлы '''
    
    time.sleep(1)
    
    wait = WebDriverWait(driver, 10)
    rows = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.debtor-documents.info-row-item tbody tr')))

    if len(rows) == 0:
        return 0, 0

    cnt_rename = 0
    cnt_need_load = 0
    for row in rows:
        # Находим кнопку скачивания в текущей строке
        button = row.find_element(By.CSS_SELECTOR, 'button')

        # Получить текст, связанный с кнопкой, убедившись, что текст чист от пробелов и спецсимволов
        text = row.find_element(By.CSS_SELECTOR, 'td').text.strip()

        have_targ_text = False
        for targ_text in ('прекращении', 'возвращении', 'приостановлении', 'қаулы атқарушылық іс жүргізуді тоқтату туралы', 'қаулы атқарушылық құжатты қайтару туралы'): #возвращении
            if targ_text in text:
                have_targ_text = True
                cnt_need_load += 1


        if have_targ_text == False:
            continue
        # До скачивания. 
        old_files = find_old_files(loading_path_docs)
        
        # Клик по кнопке для скачивания
        button.click()

        # Пауза, если требуется
        time.sleep(2.2)

        cnt_rename += rename_file(loading_path_docs, old_files, eid_iter)

        if cnt_need_load < cnt_rename:
            print(f'Переменовал БОЛЬШЕ, Кол-во должно быть скачено: {cnt_need_load} скачено: {cnt_rename}')
        elif cnt_need_load > cnt_rename:
            print(f'Переменовал Меньше Кол-во должно быть скачено: {cnt_need_load} скачено: {cnt_rename}')

    return cnt_rename, len(rows)


def find_old_files(dowlend_path):
    time.sleep(0.2)
    files = []
    dowlend_path = Path(dowlend_path)        
    files = list(dowlend_path.glob('*.pdf'))
    
    return files


def rename_file(dowlend_path, old_files, eid_iter):
    dowlend_path = Path(dowlend_path)
    new_files = []

    time.sleep(0.1) #Без этого не успевает замечать что появились новые файлы. 0.4 сек минимум
    
    t_start = time.time()
    while True:
        files_now = list(dowlend_path.glob('*.pdf'))
        
        if len(old_files) < len(files_now):
            new_files = [i for i in files_now if not i in old_files]
            break
        if time.time() - t_start > 8:
            print('Не найдено файла')
            time.sleep(0.1)
            break
        

#     print('new_files', len(new_files))
    cnt_rename = 0
    for f_n in new_files:
        for i in range(1, 1000): #Создаю версию файла. Если с таким названием уже есть. 
            add_text = f'eid_{int(eid_iter)}__v{i}'
            rename_f_n = f_n.with_name(add_text + f_n.stem + f_n.suffix)
            
            if not rename_f_n.exists():
                f_n.rename(rename_f_n)
                break
        
        
        cnt_rename += 1
    
    print('Кол-во изменений', cnt_rename)
    return cnt_rename      

    
    
def close_driver(driver, t_start_sec, try_while):
    close_error = False
    t_end_sec = time.time()
    
    try: 
        time.sleep(5)
        driver.close()
    except:
        close_error = True
    try: 
        time.sleep(2)
        driver.quit()
    except:
        pass
    driver = None
    
    print(f'Закрытие через {t_end_sec - t_start_sec} ceк' )
    break_flag = False

    sleep_time = 3
    time.sleep(sleep_time)
'''
def close_driver(driver, t_start_sec, try_while):
    t_end_sec = time.time()
    
    try: 
        time.sleep(2)
        driver.quit()
    except:
        pass
    driver = None
    
    print(f'Закрытие через {t_end_sec - t_start_sec} ceк' )
    break_flag = False

    time.sleep(5)
    '''

    

def date_to_str(date_):
    day_   = str(date_.day)
    month_ = str(date_.month)
    year_ = str(date_.year)
    if len(day_) == 1:
        day_ = '0' + day_
    if len(month_) == 1:
        month_ = '0' + month_
    return f'{day_}.{month_}.{year_}'

def wait_load_web(driver, max_sec_wait=10):
    driver.implicitly_wait(0.1)
    t_start = time.time()
    refresh_driver = False
    while True:
        time.sleep(0.1)
        t_now = time.time()
        
        if t_now - t_start >= max_sec_wait:
            print('Слишком долгая загрузка сайта. Обновляю странцу.')
            refresh_driver = True
            driver.refresh()
            break
        
        try:
            driver.find_element(By.CSS_SELECTOR, '.v-progress-linear__buffer')
            continue
        except:
            break
        
    driver.implicitly_wait(10)
    return refresh_driver
    
def main():
    pass
    
# main()


# In[9]:


# https://stackoverflow.com/questions/35772001/how-to-handle-a-signal-sigint-on-a-windows-os-machine


# In[10]:


df_main_.rename(columns={'Номер исполнительного производства':'f258', 'ИИН/БИН должника':'ИИН'}, inplace=True)


# In[11]:


list_trys = []



break_flag = False


for name_login in dict_log_pas.keys():
    if break_flag:
        break
        
    print()
    print('***', name_login, '***')

    log_ais_df = df_main_.copy()

    
    log_ais_df.sort_values(by=['ИИН'])
    log_ais_df.reset_index(drop=True, inplace=True)    
    status_pars = {'new_driver':True, 'new_log':True, 'set_web_list_targ':True, 'set_iin':True, 'set_100':True}

    for index, row in tqdm(log_ais_df.iterrows(), total=log_ais_df.shape[0], mininterval=120):
        if break_flag:
            break
            
        
        iin_iter = row['ИИН']
        f258_iter = row['f258']
        eid_iter = row['Номер_строки_google_t']
        count_men_iter = row['Судебный исполнитель']
        travel_restriction_flag = 0
        cnt_rename = 0
    
    
        print(iin_iter)
        break_while = False
        try_while_max = 15
        try_while = 0
        try_docs_load = 0
        try_docs_load_max = 3
        
        
        while (not break_while) and (not break_flag) and (try_while_max >= try_while):
            try_while += 1
            
            # Задаю исходные значения параметов. Чтобы при ошибке не возникало ситуации когда
            # из-за 15 попыток у меня проставилось предыдущее значение. 
            
            count_line = 'error'
            find_f258_iter = 0
            travel_restriction_flag = 0
            notarial_restriction_flag = 0
            restrictive_measures = '-'
            cnt_download = 0
            
            try:
                t_start_sec = time.time()
                              
                if status_pars['new_driver']:
                    driver = to_site(loading_path_docs)
                    status_pars['new_driver'] = False
                    
                if len(driver.window_handles) > 1:
                    break_flag = True
                    print('Добавлена новая вкладка. Это условие остановки цикла.')
                
                if status_pars['new_log']:# or login_new:
                    write_login(driver, name_login)
                    status_pars['new_log'] = False
                    
                if status_pars['set_web_list_targ']:
                    to_targer_sheet_1(driver)
                    to_targer_sheet_2(driver)
                    status_pars['set_web_list_targ'] = False
                    status_pars['set_100'] = True
                    
                
                if status_pars['set_iin']:
                    delite_filter(driver)
                    iin_insert(driver, iin_iter)
                    buttom_find(driver)
                    

                refresh_driver = wait_load_web(driver) # Ожидание если грузит.
                if refresh_driver:
                    status_pars['set_web_list_targ'] = True
                    continue
                
                count_line = count_data_line(driver)
                
                
                if count_line == '–':
                    login_new = False
                    break

                if status_pars['set_100']:
                    set_100_row(driver)
                    status_pars['set_100'] = False
                    count_line = count_data_line(driver)    #Если расширяли диапазон. То смотрим новое значение. 

                find_f258_iter = find_ispol_on_f258(driver, f258_iter)
                if not find_f258_iter:
                    login_new = False
                    break
                    
                refresh_driver = wait_load_web(driver) # Ожидание если грузит.
                if refresh_driver:
                    status_pars['set_web_list_targ'] = True
                    continue
    
#                 old_files    = find_old_files(loading_path_docs)    

                time.sleep(try_docs_load)
                

                cnt_download, cnt_docs = download_docs(driver, loading_path_docs, eid_iter)
#                 cnt_rename   = rename_file(loading_path_docs, old_files, eid_iter)
                if cnt_docs == 0:
                    try_docs_load += 1

                
                close_detail_window(driver)
                login_new = False
                
                if 0 < try_docs_load and try_docs_load <= try_docs_load_max:
                    print('Пусто обновляем браузер', iin_iter, cnt_docs)
                    driver.refresh()
                    status_pars['set_web_list_targ'] = True
                    continue
                
                break                
            except Exception as e:

                print(datetime.now(), 'Время срабатывания ошибки')
                exc_type, exc_value, exc_traceback = sys.exc_info()
                tb = traceback.extract_tb(exc_traceback)
                str_error = e
                if tb:
                    # The first item in the traceback list is the location of the error
                    file, line_number, function, text = tb[0]
                    str_error = f"{e}\n Error occurred in file {file}, line {line_number}: {text}"
                print(str_error)
                error_list.append(str_error)
                
#                 break_flag = True # TODO
#                 break # TODO
                
                try:
                    if try_while <= 2:
                        driver.refresh()
                        status_pars = {'new_driver':False, 'new_log':False, 'set_web_list_targ':True, 'set_iin':True, 'set_100':True}

                        time.sleep(1)
                    elif try_while <= 4:
                        status_pars = {'new_driver':False, 'new_log':True, 'set_web_list_targ':True, 'set_iin':True, 'set_100':True}
                        time.sleep(1)
                    else:
                        assert 0, f'Создано новое окно. После 4х обновления, попытка {try_while}'
                except:
                    close_driver(driver, t_start_sec, try_while)
                    status_pars = {'new_driver':True, 'new_log':True, 'set_web_list_targ':True, 'set_iin':True, 'set_100':True}

        # Сбор итога
        result_row = [
            name_login,
            index,
            eid_iter,
            count_men_iter,
            iin_iter,
            f258_iter,
            count_line,
            find_f258_iter,
            cnt_download
                    ]

        result_list.append(result_row)
          
        if index % 5 == 0:
            new_table = (index == 0) and len(df_res_old) == 0
            save_data(result_list_batch=result_list,  conn=conn_local_db, new_table=new_table, table_name = 'cancels_')
            result_list = []
        

query = ''' select * from cancels_ '''
df_sql = pd.read_sql_query(query, conn_local_db)
conn_local_db.close()

col = ['Имя портфеля', 'index', 'eid', 'ЧСИ', 'inn', 'f258', 'найдено строк', 'найдено дело', 'Кол-во файлов']
df_sql.columns = col
df_temp_itog = df_sql


# # Получаю данные по каждому файлу

# In[15]:



def find_text(text):
    
    text = text.lower()
    if 'установил' in text and 'постановил' in text:
        result = re.search(r'установил(.*?)постановил', text, re.DOTALL)

        if result:
            text_between_patterns = result.group(1)
            return text_between_patterns.replace('\n', ' ')
    else:
        return ''
    
def find_date(text):
    text = text.lower()
    
    pattern = r"(.*?)(\d{2}\.\d{2}\.\d{4})"
    match = re.search(pattern, text,  re.DOTALL)
    
    if match:
        text_before_date = match.group(1)
#         print('text_before_date 1', text_before_date)
        running_title = ''''\n*стороны исполнительного производства могут ознакомиться с материалами исполнительного производства, зарегистрировавшись на портале аис оип \nпо адресу https://aisoip.adilet.gov.kzпостановление\n'''
        text_before_date = [i for i in text_before_date.split('\n') if i not in [' ', '']][-1] #Убираю колонтитул. Оставляю только последнее предложение.
#         print('text_before_date 2', text_before_date)
        date = match.group(2)
        
        return text_before_date, date
    else:
        return '', ''
    

def extract_paragraphs_from_pdf(file_path):
    paragraphs = ''
    
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        
        for page in reader.pages:
            text = page.extract_text()
            paragraphs += '\n' + text
    
    return paragraphs

def add_type_finish(s):
    s = s.replace(' ', '')
    if 'подпунктом7' in s:
        return 'погасил'
    elif "подпунктом5-1" in s:
        return "судебное банкротство"
    elif "подпунктом5-2" in s:
        return "внесудебное банкротство"
    elif 'подпунктом5' in s:
        return 'отмена'
    elif 'подпунктом3' in s:
        return "умерший"



eid_texts = defaultdict(list)
# path_docs = Path('C:/Users/user/Задачи/АИС ОИП - отмены, возражения/load/2023-06-22___11-27/docs')

path_docs = [item for item in Path(loading_path_docs).iterdir()]
path_docs.sort()

for item in tqdm(path_docs):
#     print(item)
    f_name = item.stem
    eid = f_name.split('__', maxsplit=1)[0]
    if 'eid_' in eid:
        eid = eid.replace('eid_', '')
    else:
        continue
    
    if item.suffix == '.pdf':
        text = extract_paragraphs_from_pdf(item)
#         print(text)
        text_target = find_text(text)
        text_before_date, date = '', ''
        if text_target != "":
            text_before_date, date = find_date(text)
            type_finish = add_type_finish(text_target)
            
            add_param = [text_target, type_finish, text_before_date, date]
            if add_param not in eid_texts[eid] and text_before_date in ['о прекращении исполнительного производства', 'о возвращении исполнительного документа', 'о приостановлении исполнительного производства']:
                eid_texts[eid].append(add_param)
    

list_itog = []
for k, v in eid_texts.items():
   print(v)
   l_add = [item for sublist in v for item in sublist]
   print(l_add)
   
   list_itog.append([k, l_add])
   
   
df_eid_texts = pd.DataFrame(list_itog)

df_list = pd.DataFrame(df_eid_texts[1].tolist())

cols_list = []
for i in range(1, 1+len(list(df_list)) // 4):
   cols_list.extend([f'Установил {i}', f'Класс {i}', f'Заголовок {i}', f'Дата {i}'])
   
df_eid_texts[cols_list] = df_list
df_eid_texts = df_eid_texts.rename(columns={0:'Уникальный', 1:'Список'})

# In[17]:


# df_eid_texts.drop([1], axis=1, inplace=True)
# df_eid_texts.rename(columns={0: 'Уникальный'}, inplace=True)


df_eid_texts.to_excel(f'{loading_path}/Отмены {dt_string}.xlsx', engine='xlsxwriter')


# In[19]:
# Класс = "погасил" приоритетный. (закидываем в Класс 1)
## Находим все колонки, начинающиеся на 'Класс'
class_columns = [col for col in df_eid_texts.columns if col.startswith('Класс')]
date_columns = [col for col in df_eid_texts.columns if col.startswith('Дата')]

## Проверяем, есть ли в строке хотя бы одно 'погасил'
mask = df_eid_texts[class_columns].apply(lambda row: 'погасил' in row.values, axis=1)

## Устанавливаем 'погасил' в 'Класс 1' там, где условие выполняется
df_eid_texts.loc[mask, 'Класс 1'] = 'погасил'

df_eid_texts["Дата 1"] = df_eid_texts[date_columns].apply(
    lambda row: pd.to_datetime(row, errors='coerce').max(),
    axis=1
)# TODO !!! 17.11.2025
# In[20]:


df_final = pd.merge(left=df_temp_itog, right=df_eid_texts, how='left', left_on = 'eid', right_on = 'Уникальный')
df_final.drop(columns='Уникальный', inplace=True)




df_final.to_excel(f'{loading_path}/Отмены Итог {dt_string}.xlsx', engine='xlsxwriter', index=False)


# # Сохраняю в "проверенные"
df_check_new = df_final[['inn', 'f258', 'Класс 1', "Дата 1"]].rename(columns={'inn':'ИИН', 'f258':'Номер исполнительного производства'})
df_сheck_ = pd.DataFrame(pd.concat([df_old_[['ИИН', 'Номер исполнительного производства', 'Класс 1', "Дата 1"]], df_check_new]), columns=['ИИН', 'Номер исполнительного производства', 'Класс 1', "Дата 1"])
df_сheck_.to_excel(path_check_iin, index=False)



# In[24]:


#save_data(result_list, name_login, loading_path, index)


# # Добавляю прошлые записи

# In[25]:


file_date = {}
for f in Path(loading_path_all).iterdir():
    
    if f.is_dir():

        p_name = f.name
        if 'new_load' in p_name:
            continue

        p_name_s = p_name.split(' ')
        p_time = p_name_s[0]

        try:
            dt = datetime.strptime(p_time, "%Y-%m-%d___%H-%M")
        except:
            continue
        file_date[f] = dt


final_file_last = ''

add_folder_sort = sorted(file_date, key=file_date.get)[::-1]
if len(add_folder_sort) > 1:
    old_add_folder = add_folder_sort[0]
    
    
    for f in Path(old_add_folder).iterdir():
        if 'Итог_union' in str(f): #Добавить union
            final_file_last = f
    if final_file_last == '':
        print('НЕТ ПРОШЛОГО ФАЙЛА Итог_union')

    
        
if final_file_last != '':
    df_last_file = pd.read_excel(final_file_last)
else:
    df_last_file = pd.DataFrame()


# Добавляю прошлвые данные.

df_union = pd.concat([df_final, df_last_file], axis=0, ignore_index=True)

df_union['inn'] = df_union['inn'].astype(str)
df_union['inn'] = df_union['inn'].apply(lambda x: (12 - len(x))*'0'+x )

# df_union = df_union.drop_duplicates(subset=['inn'])

if len(df_sql) >= df_main_start_len-1:
    # Сохраняю объедененный файл
    df_union.to_excel(f'{loading_path}/Отмены Итог_union {dt_string}.xlsx', engine='xlsxwriter', index=False)

    # Переименовываю папку. 
    current_folder = Path(loading_path)
    
    dt_string = now.strftime("%Y-%m-%d___%H-%M")
    new_folder = current_folder.parent/f'{dt_string}'
    
    current_folder.rename(new_folder)


в последнем скаченном АСИОИП файле нет "итога"


FileNotFoundError: [Errno 2] No such file or directory: ''